# How to use APIs that are openai-compatible (Together, OpenAI, etc.)

In [3]:
from flow_eval import OpenAIModel, EvalInput, AsyncFlowJudge

from flow_eval.metrics import RESPONSE_FAITHFULNESS_5POINT
import json
import asyncio

import nest_asyncio
nest_asyncio.apply()

# model = OpenAIModel(exec_async=True)

model = OpenAIModel(
    _model_id="meta-llama/Llama-3-8b-chat-hf",
    base_url="https://api.together.xyz/v1",
    key_var="TOGETHER_API_KEY",
    exec_async=True
)

# Initialize the judge
faithfulness_judge = AsyncFlowJudge(
    metric=RESPONSE_FAITHFULNESS_5POINT,
    model=model
)

with open("/home/bernardo/repos/flow-eval/examples/sample_data/csr_assistant.json", "r") as f:
    data = json.load(f)

# Create a list of inputs and outputs
inputs_batch = [
    [
        {"query": sample["query"]},
        {"context": sample["context"]},
    ]
    for sample in data
]
outputs_batch = [{"response": sample["response"]} for sample in data]

# Create a list of EvalInput
eval_inputs_batch = [EvalInput(inputs=inputs, output=output) for inputs, output in zip(inputs_batch, outputs_batch)]

# Run the evaluation
async def main():
    results = await faithfulness_judge.async_batch_evaluate(eval_inputs_batch, save_results=False)
    return results

results = asyncio.run(main())

print(results)

INFO:flow_eval.models.openai:Successfully initialized OpenAI!
INFO:flow_eval.models.openai:Initiating batched OpenAI async requests
100%|██████████| 6/6 [00:00<00:00, 87685.80it/s]
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


[EvalOutput(feedback='The generated response is mostly consistent with the provided context. The response accurately explains the solution to the customer\'s issue with pushing large files to their Git repository, using Git Large File Storage (LFS). The steps provided are clear and concise, and they are directly supported by the context. The only minor inconsistency is the inclusion of the sentence "By using Git LFS, you\'ll be able to push files larger than GitHub\'s 100 MB limit, as LFS will handle them appropriately." which is not explicitly mentioned in the context, but it is a logical conclusion based on the information provided. Overall, the response is well-structured and easy to follow, making it a reliable solution for the customer\'s issue.', score=4), EvalOutput(feedback='The generated response is mostly inconsistent with the provided context. While it includes some information from the context, it introduces a substantial amount of hallucinated or fabricated details that de